### 사전 준비

In [ ]:
!pip install transformers==4.29 torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.1 MB/s eta 0:00:00


In [ ]:
!pip install pycocoevalcap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 8.7 MB/s eta 0:00:00


In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import json
import os
from PIL import Image
import torch
import time

In [ ]:
class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], text=item["text"], padding="max_length", return_tensors="pt")

        encoding = {k:v.squeeze() for k,v in encoding.items()}
        return encoding

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### preprocess 준비

In [ ]:
from transformers import  AutoProcessor
from transformers import AutoModelForCausalLM, Swin2SRImageProcessor
import numpy as np
processor = AutoProcessor.from_pretrained('/content/drive/MyDrive/재원/model/caption/git/preprocessor')
pro_sr = Swin2SRImageProcessor.from_pretrained('/content/drive/MyDrive/재원/eval/sr/preprocessor')
model_sr = torch.load('/content/drive/MyDrive/재원/eval/sr/swinsr.pt')

### 옵션

In [ ]:
option = {
    'start': 0,
    'num': 5000,
    'batch_size':8,
}

### 데이터셋 준비 path는 알아서 잘 조정하기 - 1000개당 3분 정도 걸림

In [ ]:
#zip 풀기
!unzip -qq '/content/drive/MyDrive/data/5000.zip' -d '/content'

In [ ]:
labelpath="/content/caption.json"
with open(labelpath, 'r',encoding = 'utf-8' or 'cp949' ) as f: # json 파일 접근
    captions = json.load(f)
import random
random.seed(777)
random.shuffle(captions)

In [ ]:
def image_list_with_sr(captions,dir,n,m): # 이미지 데이터 가져오기
    imagelist=[]
    for i in range(n,n+m):
        path = dir+'/'+captions[i]['image']
        image = Image.open(path)
        image = super_reso(image) if image.size[0]<50 or image.size[1]<100 else image
        imagelist.append(image)
    return imagelist

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_sr.to(device)
def super_reso(image):
    inputs = pro_sr(image, return_tensors="pt").to(device)

    # forward pass
    with torch.no_grad():
        outputs = model_sr(**inputs)

    output = outputs.reconstruction.data.squeeze().cpu().float().clamp_(0, 1).numpy()
    output = np.moveaxis(output, source=0, destination=-1)
    output = (output * 255.0).round().astype(np.uint8)
    return Image.fromarray(output)

In [ ]:
ImageList = image_list_with_sr(captions,'/content/image',option['start'],option['num'])
model_sr.to('cpu') # 5000장에서 약 2분 소요

Swin2SRForImageSuperResolution(
  (swin2sr): Swin2SRModel(
    (first_convolution): Conv2d(3, 60, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (embeddings): Swin2SREmbeddings(
      (patch_embeddings): Swin2SRPatchEmbeddings(
        (projection): Conv2d(60, 60, kernel_size=(1, 1), stride=(1, 1))
        (layernorm): LayerNorm((60,), eps=1e-05, elementwise_affine=True)
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Swin2SREncoder(
      (stages): ModuleList(
        (0-3): 4 x Swin2SRStage(
          (layers): ModuleList(
            (0-5): 6 x Swin2SRLayer(
              (attention): Swin2SRAttention(
                (self): Swin2SRSelfAttention(
                  (continuous_position_bias_mlp): Sequential(
                    (0): Linear(in_features=2, out_features=512, bias=True)
                    (1): ReLU(inplace=True)
                    (2): Linear(in_features=512, out_features=6, bias=False)
                  )
                  (query):

In [ ]:
data = [{'text':captions[i]['label'],'image':ImageList[i]} for i in range(option['start'],option['start']+option['num'])]

In [ ]:
train_dataset = ImageCaptioningDataset(data[:int(0.8*option['num'])], processor)
val_dataset = ImageCaptioningDataset(data[int(0.8*option['num']):], processor)
train_dataloader = DataLoader(train_dataset,shuffle=False,batch_size = option['batch_size'])
val_dataloader = DataLoader(val_dataset,shuffle=False,batch_size = option['batch_size'])

In [ ]:
num_train = int(0.8*option['num']/option['batch_size'])
num_test = int(0.2*option['num']/option['batch_size'])

### 성능 평가 코드

In [ ]:
def gen_captions(captions,filename):
    gen = []
    for i in range(len(captions)):
        gen.append({'image_id': i+1, 'caption': captions[i]})

    with open(filename,'w') as f:
        json.dump(gen,f)

In [ ]:
def make_gt(captions,filename):
    annotations=[]
    images = []
    for i in range(len(captions)):
        annotations.append({'image_id': i+1, 'caption': captions[i],'id': i+1})
        images.append({'id':i+1})
    gt_captions ={
        'annotations': annotations,
        'images': images
    }
    with open(filename,'w') as f:
        json.dump(gt_captions,f)

In [ ]:
import json
import os
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap

def coco_caption_eval(annotation_file, results_file):

    coco = COCO(annotation_file)
    coco_result = coco.loadRes(results_file)

    coco_eval = COCOEvalCap(coco, coco_result)
    coco_eval.evaluate()

    return coco_eval

### layer Freezing

In [ ]:
mode = "microsoft/git-base"
model = AutoModelForCausalLM.from_pretrained(mode)

In [ ]:
#frozen_layers = ['embeddings','image_encoder']
frozen_layers=[]

# 레이어 동결
if len(frozen_layers)==0:
    print("NO Freeze!!!!")
else:
    for name, param in model.named_parameters():
          if any(frozen_layer in name for frozen_layer in frozen_layers):
                param.requires_grad = False
                print(f'{name} is freeze!!!')

### finetuning 진행

In [ ]:
import torch
val_cpath = '/content/drive/MyDrive/재원/captions/sr/val'
val_rpath = '/content/drive/MyDrive/재원/captions/sr/val_real.json'
train_hist=[]
val_hist = []
train_iter_hist=[]
val_iter_hist=[]
val_eval = []

In [ ]:
make_gt([item['text'] for item in data][int(0.8*option['num']):],val_rpath)#val의 진짜 caption

In [ ]:
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
lr = 1e-5
E = 30

optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.05)
scheduler = CosineAnnealingLR(optimizer, T_max=E)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

torch.manual_seed(777)
for epoch in range(E):
    model.train()
    Loss = 0

    train_caption = []
    for idx, batch in enumerate(train_dataloader):
        model.train()
        input_ids = batch.pop("input_ids").to(device)
        pixel_values = batch.pop("pixel_values").to(device)
        outputs = model(input_ids=input_ids,pixel_values=pixel_values, labels=input_ids)
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #성능을 보기 위한 작업
        Loss+=loss.tolist()
        train_iter_hist.append(loss.tolist())


    train_hist.append(Loss/len(train_dataloader))


    #validation진행

    val = 0
    val_caption =[]
    with torch.no_grad():
        model.eval()
        for idx, batch in enumerate(val_dataloader):
            input_ids = batch.pop("input_ids").to(device)
            pixel_values = batch.pop("pixel_values").to(device)
            outputs = model(input_ids=input_ids,pixel_values=pixel_values, labels=input_ids)

            #성능을 보기위한 작업
            val_caption+=processor.batch_decode(model.generate(pixel_values=pixel_values,max_length = 100),skip_special_tokens=True)
            val+=outputs.loss.tolist()
            val_iter_hist.append(loss.tolist())

    val_hist.append(val/len(val_dataloader))

    #checkpoint
    if val_hist[-1]==min(val_hist):
        torch.save(model,'/content/drive/MyDrive/재원/model/caption/git/git_25000_all.pt')

    #Epoch 출력
    print("Epoch {}회차 - val_Loss:{}, ".format(epoch+1,val*option['batch_size']/(0.2*option['num'])))

    #epoch의 caption들 저장 및 성능 출력을 위한 코드들
    gen_captions(val_caption,val_cpath+'/'+str(epoch+1)+'.json')
    val_eval.append(coco_caption_eval(val_rpath,val_cpath+'/'+str(epoch+1)+'.json').eval.items())

In [ ]:
import json
total_list =[train_hist,[train_iter_hist,val_iter_hist],val_hist,[dict(item.mapping) for item in val_eval]]
with open('/content/drive/MyDrive/재원/eval/loss/git_25000_all_plot.json','w') as f:
    json.dump(total_list,f)